In [1]:
import pandas as pd
from IPython.display import display, HTML
import sqlite3
import folium
from folium.plugins import HeatMap

In [2]:
df = pd.read_csv("../Data/inmuebles-sample.csv")

In [3]:
# Ver nombres de las columnas
print(df.columns.tolist())

['website', 'url', 'reference', 'country', 'province', 'location', 'title', 'description', 'price', 'operation', 'property_type', 'rooms', 'baths', 'area', 'floor', 'elevator', 'outside', 'floor.1', 'images', 'latitude', 'longitude', 'dealer', 'dealer_url', 'dealer_is_professional']


In [4]:
# Información general del dataframe
df.info()

# También puedes ver si hay columnas sin nombre
print(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   website                 1000 non-null   object 
 1   url                     1000 non-null   object 
 2   reference               943 non-null    object 
 3   country                 1000 non-null   object 
 4   province                1000 non-null   object 
 5   location                1000 non-null   object 
 6   title                   1000 non-null   object 
 7   description             1000 non-null   object 
 8   price                   994 non-null    float64
 9   operation               1000 non-null   object 
 10  property_type           1000 non-null   object 
 11  rooms                   433 non-null    float64
 12  baths                   475 non-null    float64
 13  area                    952 non-null    float64
 14  floor                   53 non-null     f

In [5]:
# Tipos de datos
print(df.dtypes)

# Valores únicos por columna (ideal para variables categóricas)
for col in df.columns:
    print(f"{col}: {df[col].nunique()} valores únicos")


website                    object
url                        object
reference                  object
country                    object
province                   object
location                   object
title                      object
description                object
price                     float64
operation                  object
property_type              object
rooms                     float64
baths                     float64
area                      float64
floor                     float64
elevator                  float64
outside                   float64
floor.1                   float64
images                      int64
latitude                  float64
longitude                 float64
dealer                     object
dealer_url                 object
dealer_is_professional       bool
dtype: object
website: 2 valores únicos
url: 846 valores únicos
reference: 789 valores únicos
country: 1 valores únicos
province: 12 valores únicos
location: 436 valores únicos
title: 

In [6]:
def mostrar_tabla_con_scroll(df, filas=100, alto_px=500):
    html = df.head(filas).to_html(classes='table table-striped', index=False)

    scroll_html = f"""
    <div style="height:{alto_px}px; overflow:auto; border:1px solid #ccc; padding:10px">
        {html}
    </div>
    """
    display(HTML(scroll_html))

# Llama a la función
mostrar_tabla_con_scroll(df, filas=100, alto_px=500)

website,url,reference,country,province,location,title,description,price,operation,property_type,rooms,baths,area,floor,elevator,outside,floor.1,images,latitude,longitude,dealer,dealer_url,dealer_is_professional
yaencontre.com,https://www.yaencontre.com/alquiler/piso/inmueble-44601-94902282,44601-94902282,es,santa-cruz-de-tenerife,"Puerto de Santiago, Santiago del Teide, Santa Cruz de Tenerife (provincia)","Piso en calle Manuel Ravelo en Puerto de Santiago, Santiago del Teide","Dream Homes Tenerife les presenta este fantástico Apartamento en alquiler de 2 Dormitorios junto a la playa en Puerto de Santiago, Complejo Seguro del Sol.\r\n\r\nDISPONIBLE DESDE OCTUBRE 2023 HASTA JUNIO 2024.\r\n\r\nIMPORTANTE: Se alquila solo por temporada, por tiempo ...",850.0,RENT,FLAT,2.0,1.0,77.0,NaN,NaN,NaN,NaN,16,28.231049,-16.839985,Dream Homes Tenerife,https://www.yaencontre.com/inmobiliarias/dream-homes-tenerife-44601,True
yaencontre.com,https://www.yaencontre.com/alquiler/piso/inmueble-44357-102244852,44357-102244852,es,santa-cruz-de-tenerife,"La Salle - Cuatro Torres, La Salud - La Salle, Santa Cruz de Tenerife, Santa Cruz de Tenerife (provincia)","Piso en plaza De Los Curtidos Hermanos Dorta, La Salle - Cuatro Torres, Santa Cruz de Tenerife","Alquiler de vivienda situada en el Residencial Los Molinos, en la trasera del Colegio de San Idelfonso de La Salle. Santa Cruz de Tenerife.\r\n\r\nEstá ubicada en la primera planta del edificio con ascensor, destaca porque es muy amplia y luminosa.\r\n\r\nSe distribuye en...",1200.0,RENT,FLAT,3.0,2.0,110.0,NaN,NaN,NaN,NaN,37,28.462198,-16.255582,Dorta Hermanos Inmobiliaria,https://www.yaencontre.com/inmobiliarias/dorta-hermanos-inmobiliaria-44357,True
yaencontre.com,https://www.yaencontre.com/alquiler/piso/inmueble-44601-98327574,44601-98327574,es,santa-cruz-de-tenerife,"Acantilados de Los Gigantes, Santiago del Teide, Santa Cruz de Tenerife (provincia)","Piso en Acantilados de Los Gigantes, Santiago del Teide","Dream Homes Tenerife le presenta este acogedor apartamento de un dormitorio en Los Gigantes. El apartamento tiene 1 dormitorio, un baño, cocina semi independiente, salón y trastero.\r\n\r\nEl apartamento está muy bien ubicado en un barrio tranquilo y cerca de todos ...",785.0,RENT,FLAT,1.0,1.0,85.0,NaN,NaN,NaN,NaN,15,28.243130,-16.839593,Dream Homes Tenerife,https://www.yaencontre.com/inmobiliarias/dream-homes-tenerife-44601,True
yaencontre.com,https://www.yaencontre.com/alquiler/piso/inmueble-44790-101478426,44790-101478426,es,santa-cruz-de-tenerife,"Costa Blanca - Las Galletas, Arona, Santa Cruz de Tenerife (provincia)","Piso en Costa Blanca - Las Galletas, Arona","SOLO CORTA TEMPORADA!\r\nDISPONIBLE DESDE EL 14.09. 2023!\r\nInmogestion Tenerife ofrece en alquiler, una vivienda en la tercera planta en el pueblo pesquero Las Galletas!\r\nLa vivienda dispone de 2 dormitorios (con armarios empotrados), 1 baño con ducha y bidet, pasi...",990.0,RENT,FLAT,2.0,1.0,59.0,NaN,NaN,NaN,NaN,22,28.009573,-16.659095,Inmogestion tenerife,https://www.yaencontre.com/inmobiliarias/inmogestion-tenerife-44790,True
yaencontre.com,https://www.yaencontre.com/alquiler/piso/inmueble-48951-97394554,48951-97394554,es,santa-cruz-de-tenerife,"Güímar, Santa Cruz de Tenerife (provincia)",Piso en Güímar,"¡Alquiler por corta temporada AGOSTO 950,00€ con garaje, está disponible de 1 al 29 de agosto de 2023\r\n\r\nCoqueto y acogedor piso con excelente ubicación a tan sólo 200 metros del mar y todos los servicios, tendrás la tranquilidad y comodidad que buscas en el lug...",950.0,RENT,FLAT,2.0,1.0,75.0,NaN,NaN,NaN,NaN,20,28.297265,-16.374269,Lucrecia Magistrati Servicios Inmobiliarios Tu Casa y Asesoramiento integral,https://www.yaencontre.com/inmobiliarias/lucrecia-magistrati-servicios-inmobiliarios-tu-casa-y-asesoramiento-integral-48951,True
pisos.com,https://www.pisos.com/alquilar/nave_industrial-matama_beade_bembrive_valadares_zamans-35052667848_101800/,35052667848.101800,es,pontevedra,"Calle Estrada De Valadares, Matamá-B

In [7]:
# Normalizar valores en la columna 'operation'
df['operation'] = df['operation'].str.lower()  # pasar todo a minúsculas para estandarizar

# Reemplazar 'venta' por 'buy' y 'alquiler' por 'rent'
df['operation'] = df['operation'].replace({
    'venta': 'buy',
    'alquiler': 'rent'
})

# Pasar todo a mayúsculas final
df['operation'] = df['operation'].str.upper()


In [8]:
print("🟠 operation:")
print(df['operation'].value_counts(), end='\n\n')

print("🟢 property_type:")
print(df['property_type'].value_counts(), end='\n\n')

print("🔵 province:")
print(df['province'].value_counts(), end='\n\n')

🟠 operation:
operation
BUY     588
RENT    412
Name: count, dtype: int64

🟢 property_type:
property_type
FLAT                332
LAND                299
pisos                91
naves                72
locales              60
NEW_CONSTRUCTION     57
garajes              56
terrenos             28
PREMISES              5
Name: count, dtype: int64

🔵 province:
province
santa-cruz-de-tenerife    314
barcelona                 299
valencia                  155
pontevedra                 98
navarra                    57
albacete                   26
malaga                     18
alava                      18
lugo                        6
caceres                     4
navarra_nafarroa            4
tarragona                   1
Name: count, dtype: int64



In [9]:
conn = sqlite3.connect(':memory:')
df.to_sql('inmuebles', conn, index=False, if_exists='replace')
# Crear cursor
cursor = conn.cursor()

In [10]:
query1 = """
SELECT operation, COUNT(*) as total
FROM inmuebles
WHERE province = 'santa-cruz-de-tenerife'
GROUP BY operation;
"""

# Ejecutar la consulta y mostrar resultados
resultados = cursor.execute(query1).fetchall()
for op, total in resultados:
    print(f"{op}: {total} propiedades")

RENT: 314 propiedades


In [11]:
# Filtrar solo pisos en alquiler en Santa Cruz de Tenerife
df_sc = df[(df['province'].str.lower() == 'santa-cruz-de-tenerife') & (df['operation'] == 'RENT')].copy()

# Revisamos qué tenemos
print(f"Total de registros: {len(df_sc)}")
print(df_sc[['price', 'area', 'rooms', 'baths', 'location', 'property_type']].info())

Total de registros: 314
<class 'pandas.core.frame.DataFrame'>
Index: 314 entries, 0 to 410
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          314 non-null    float64
 1   area           314 non-null    float64
 2   rooms          274 non-null    float64
 3   baths          314 non-null    float64
 4   location       314 non-null    object 
 5   property_type  314 non-null    object 
dtypes: float64(4), object(2)
memory usage: 17.2+ KB
None


In [12]:
# Mapa centrado en Santa Cruz
mapa = folium.Map(location=[df_sc['latitude'].mean(), df_sc['longitude'].mean()], zoom_start=12)

# Agregar capa de calor
heat_data = [[row['latitude'], row['longitude'], row['price']] for idx, row in df_sc.iterrows()]
HeatMap(heat_data).add_to(mapa)

mapa

In [13]:
# Guardar el mapa en formato HTML en la carpeta Outputs
mapa.save("../Outputs/heatmaptenerife.html")

In [14]:
# Crear variable auxiliar de precio por m²
df_sc['price_per_m2'] = df_sc['price'] / df_sc['area']

# Eliminar filas con nulos en variables clave
df_modelo = df_sc.dropna(subset=['price', 'area', 'rooms', 'baths'])

In [15]:
# Calcular precio/m2 por zona
precio_por_m2 = df_sc.groupby('location')['price_per_m2'].mean().sort_values(ascending=False)

print("Precio promedio por metro cuadrado:")
print(precio_por_m2)

Precio promedio por metro cuadrado:
location
Adeje ciudad, Adeje, Santa Cruz de Tenerife (provincia)                                26.071429
Costa Adeje, Adeje, Santa Cruz de Tenerife (provincia)                                 24.663774
Los Cristianos - Playa de las Américas, Arona, Santa Cruz de Tenerife (provincia)      24.185393
Acantilados de Los Gigantes, Santiago del Teide, Santa Cruz de Tenerife (provincia)    20.300654
Playa de San Juan, Guía de Isora, Santa Cruz de Tenerife (provincia)                   20.178571
                                                                                         ...    
Araya, Candelaria, Santa Cruz de Tenerife (provincia)                                   7.692308
Tamaimo-Arguayo, Santiago del Teide, Santa Cruz de Tenerife (provincia)                 7.518182
Tacoronte - Los Naranjeros, Tacoronte, Santa Cruz de Tenerife (provincia)               7.142857
Guía de Isora interior, Guía de Isora, Santa Cruz de Tenerife (provincia)         